In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


In [2]:
categorical=pd.read_csv(r"C:\Users\torra\IH-Labs\lab-comparing-regression-models\categorical.csv")
numerical=pd.read_csv(r"C:\Users\torra\IH-Labs\lab-comparing-regression-models\file_numerical.csv")

In [3]:
data=pd.concat([categorical,numerical],ignore_index=True)
data

,customer,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,...,Unnamed: 0,customer_lifetime_value,effective_to_date,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,BU79786,Washington,No,Basic,Bachelor,Employed,F,Suburban,Married,Corporate Auto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,QZ44356,Arizona,No,Extended,Bachelor,Unemployed,F,Suburban,Single,Personal Auto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AI49188,Nevada,No,Premium,Bachelor,Employed,F,Suburban,Married,Personal Auto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WW63253,California,No,Basic,Bachelor,Unemployed,M,Suburban,Married,Corporate Auto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HB64268,Washington,No,Basic,Bachelor,Employed,M,Rural,Single,Personal Auto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9129.0,23405.987980,2011-02-10,71941.0,73.0,18.0,89.0,0.0,2.0,198.234764
18264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9130.0,3096.511217,2011-02-12,21604.0,79.0,14.0,28.0,0.0,1.0,379.200000
18265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9131.0,8163.890428,2011-02-06,0.0,85.0,9.0,37.0,3.0,2.0,790.784983
18266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9132.0,7524.442436,2011-02-03,21941.0,96.0,34.0,3.0,0.0,3.0,691.200000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   customer                       9134 non-null   object 
 1   state                          9134 non-null   object 
 2   response                       9134 non-null   object 
 3   coverage                       9134 non-null   object 
 4   education                      9134 non-null   object 
 5   employmentstatus               9134 non-null   object 
 6   gender                         9134 non-null   object 
 7   location_code                  9134 non-null   object 
 8   marital_status                 9134 non-null   object 
 9   policy_type                    9134 non-null   object 
 10  renew_offer_type               9134 non-null   object 
 11  sales_channel                  9134 non-null   object 
 12  vehicle_class                  9134 non-null  

In [5]:
data.isna().sum()

customer                         9134
state                            9134
response                         9134
coverage                         9134
education                        9134
employmentstatus                 9134
gender                           9134
location_code                    9134
marital_status                   9134
policy_type                      9134
renew_offer_type                 9134
sales_channel                    9134
vehicle_class                    9134
vehicle_size                     9134
Unnamed: 0                       9134
customer_lifetime_value          9134
effective_to_date                9134
income                           9134
monthly_premium_auto             9134
months_since_last_claim          9134
months_since_policy_inception    9134
number_of_open_complaints        9134
number_of_policies               9134
total_claim_amount               9134
dtype: int64

We drop the column "Unnamed"

In [6]:
data['Unnamed: 0'].unique()

array([      nan, 0.000e+00, 1.000e+00, ..., 9.131e+03, 9.132e+03,
       9.133e+03])

In [7]:
data=data.drop(columns='Unnamed: 0')


In [8]:

data=data.drop(columns='customer')

In [9]:
data=data.drop(columns='effective_to_date')

We make the requiered modifications in the column coverage

In [10]:
data["coverage"].mode()

0    Basic
Name: coverage, dtype: object

In [11]:
data["coverage"]=data["coverage"].fillna(value="Basic")

In [12]:
data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})

In [13]:
data["coverage"].unique()

array([0, 1, 2], dtype=int64)

We replace the missing values of the column "state" with the mode

In [14]:
data["state"].mode()

0    California
Name: state, dtype: object

In [15]:
data["state"]=data["state"].fillna(value='California')

We check the column "response", to see how to treat the missing values.

In [16]:
data['response'].unique()

array(['No', 'Yes', nan], dtype=object)

We are going to fill the missing values in the column "response" with Unknown

In [17]:
data["response"]=data["response"].fillna(value='Unknown')

We check the column "education", to see how to treat the missing values.

In [18]:
data["education"].unique()

array(['Bachelor', 'College', 'Master', 'High School or Below', 'Doctor',
       nan], dtype=object)

We are going to fill the missing values in the column "education" with Unknown

In [19]:
data["education"]=data["education"].fillna(value='Unknown')

We check the column "employmentstatus", to see how to treat the missing values.

In [20]:
data["employmentstatus"].unique()

array(['Employed', 'Unemployed', 'Medical Leave', 'Disabled', 'Retired',
       nan], dtype=object)

We are going to fill the missing values in the column "employmentstatus" with Unknown

In [21]:
data["employmentstatus"]=data["employmentstatus"].fillna(value='Unknown')

We check the column "gender", to see how to treat the missing values.

In [22]:
data["gender"].unique()

array(['F', 'M', nan], dtype=object)

We replace the missing values of the column "gender" with the mode

In [23]:
data["gender"].mode()

0    F
Name: gender, dtype: object

In [24]:
data["gender"]=data["gender"].fillna(value='F')

We check the column "location_code", to see how to treat the missing values.

In [25]:
data["location_code"].unique()

array(['Suburban', 'Rural', 'Urban', nan], dtype=object)

We replace the missing values of the column "location_code" with the mode

In [26]:
data["location_code"].mode()

0    Suburban
Name: location_code, dtype: object

In [27]:
data["location_code"]=data["location_code"].fillna(value='Suburban')

We check the column "marital_status ", to see how to treat the missing values.

In [28]:
data["marital_status"].unique()

array(['Married', 'Single', 'Divorced', nan], dtype=object)

We are going to fill the missing values in the column "marital_status" with Unknown

In [29]:
data["marital_status"]=data["marital_status"].fillna(value='Unknown')

In [30]:
data["marital_status"].unique()

array(['Married', 'Single', 'Divorced', 'Unknown'], dtype=object)

We check the column "policy_type", to see how to treat the missing values.


In [31]:
data["policy_type"].unique()

array(['Corporate Auto', 'Personal Auto', 'Special Auto', nan],
      dtype=object)

In [32]:
data["policy_type"].value_counts()

policy_type
Personal Auto     6788
Corporate Auto    1968
Special Auto       378
Name: count, dtype: int64

Taking in to consideration the small amount of "Special Auto" category, we are going to merge it with the nans. We are going to create a new category called "others"

In [33]:
data["policy_type"]=data["policy_type"].fillna(value="other")

In [34]:
data["policy_type"] = data["policy_type"].map({"Personal Auto" : "P", "Corporate Auto" : "C", "Special Auto" : "S","other":"O"})

In [35]:
data["policy_type"].unique()

array(['C', 'P', 'S', 'O'], dtype=object)

We check the column "renew_offer_type", to see how to treat the missing values.

In [36]:
data["renew_offer_type"].unique()

array(['Offer1', 'Offer3', 'Offer2', 'Offer4', nan], dtype=object)

In [37]:
data["renew_offer_type"]=data["renew_offer_type"].fillna(value="other")

We check the column "sales_channel", to see how to treat the missing values.

In [38]:
data["sales_channel"].unique()

array(['Agent', 'Call Center', 'Web', 'Branch', nan], dtype=object)

In [39]:
data["sales_channel"].value_counts()

sales_channel
Agent          3477
Branch         2567
Call Center    1765
Web            1325
Name: count, dtype: int64

In [40]:
data["sales_channel"]=data["sales_channel"].fillna(value="other")

We check the column "vehicle_class", to see how to treat the missing values.

In [41]:
data["vehicle_class"].unique()

array(['Two-Door Car', 'Four-Door Car', 'SUV', 'Luxury SUV', 'Sports Car',
       'Luxury Car', nan], dtype=object)

In [42]:
data["vehicle_class"].value_counts()

vehicle_class
Four-Door Car    4621
Two-Door Car     1886
SUV              1796
Sports Car        484
Luxury SUV        184
Luxury Car        163
Name: count, dtype: int64

We are going to group Sports Car, Luxury SUV and Luxury Car in a single category called "Luxury".
All the nan values are going to be transformed into a category called "other"

In [43]:
data["vehicle_class"]=data["vehicle_class"].fillna(value="other")

In [44]:
data["vehicle_class"] = data["vehicle_class"].map({"Four Door Car":"FDC","Two-Door Car":"TDC","SUV":"SUV","Sports Car":"Luxury","Luxury SUV":"Luxury","Luxury Car":"Luxury","other":"O"})

In [45]:
data["vehicle_class"].value_counts()

vehicle_class
O         9134
TDC       1886
SUV       1796
Luxury     831
Name: count, dtype: int64

We check the column "vehicle_size", to see how to treat the missing values.


In [46]:
data["vehicle_size"].unique()

array(['Medsize', 'Small', 'Large', nan], dtype=object)

In [47]:
data["vehicle_size"].mode()

0    Medsize
Name: vehicle_size, dtype: object

In [48]:
data["vehicle_size"]=data["vehicle_size"].fillna(value="Medsize")

We check the column "customer_lifetime_value", to see how to treat the missing values.

In [49]:
data["customer_lifetime_value"].unique()

array([        nan, 2763.519279, 6979.535903, ..., 8163.890428,
       7524.442436, 2611.836866])

In [50]:
data["customer_lifetime_value"].mode()

0       2004.350666
1       2227.072755
2       2248.449633
3       2300.691547
4       2359.423599
           ...     
213    26197.414980
214    27789.692350
215    33473.349460
216    34611.378960
217    41787.903430
Name: customer_lifetime_value, Length: 218, dtype: float64

In [51]:
data["customer_lifetime_value"]=data["customer_lifetime_value"].fillna(value="2004.350666")

We check the column "income ", to see how to treat the missing values.

In [52]:
data["income"].unique()

array([   nan, 56274.,     0., ..., 66367., 71941., 21941.])

In [53]:
data["income"].mean()

37657.38000875848

In [54]:
data["income"]=data["income"].fillna(value="37657.38000875848")

We check the column "monthly_premium_auto", to see how to treat the missing values.

In [55]:
data["monthly_premium_auto"].unique()

array([ nan,  69.,  94., 108., 106.,  73.,  67., 101.,  71.,  93., 110.,
        70.,  64.,  72.,  74.,  79.,  61., 111., 139.,  80.,  63., 237.,
        65.,  62., 117.,  91.,  90.,  66.,  99., 103., 172.,  92., 104.,
       113., 124., 105., 100., 112.,  97.,  95., 102., 186.,  77.,  96.,
        76., 119.,  88., 109., 107., 130.,  85.,  98., 133.,  86.,  78.,
        83.,  68., 192., 118.,  89., 140., 127., 168., 126.,  82., 115.,
       217., 114., 138.,  81., 255., 150., 123., 116., 190., 159.,  84.,
       152., 136., 245., 135., 128., 225., 132., 153., 121., 154., 161.,
       156., 171.,  87., 244., 183., 176., 230., 129., 177., 196., 131.,
       276., 253., 137., 180., 203., 147., 165., 198., 125., 193., 271.,
       188., 142., 206., 182., 189., 160., 295., 210., 134., 239., 256.,
       173., 285., 148., 197., 199., 214., 175., 167., 194., 164., 181.,
       222., 215., 283., 169., 248., 223., 273., 151., 213., 145., 185.,
       287., 297., 201., 149., 146., 235., 268., 21

In [56]:
data["monthly_premium_auto"].mean()

93.21929056273265

In [57]:
data["monthly_premium_auto"]=data["monthly_premium_auto"].fillna(value="93.21929056273265")

We check the column "months_since_last_claim ", to see how to treat the missing values.

In [58]:
data["months_since_last_claim"].unique()

array([nan, 32., 13., 18., 12., 14.,  0., 17., 23., 27.,  9., 21., 11.,
       31.,  8.,  4., 30., 22., 15., 33.,  1., 19., 28., 10.,  7.,  3.,
        5., 20., 24., 29., 34.,  6., 35.,  2., 16., 26., 25.])

In [59]:
data["months_since_last_claim"].mean()

15.097000218962119

In [60]:
data["months_since_last_claim"]=data["months_since_last_claim"].fillna(value="93.21929056273265")

We check the column "months_since_policy_inception ", to see how to treat the missing values.

In [61]:
data["months_since_policy_inception"].unique()

array([nan,  5., 42., 38., 65., 44., 94., 13., 68.,  3.,  7., 87., 82.,
       25., 10., 50., 59.,  1., 21., 49., 91., 47., 64., 54.,  6., 14.,
       56., 17., 40., 62., 86., 79., 15., 32., 28., 16.,  4., 57., 77.,
       95., 26., 53., 12., 98., 36., 71., 60., 20.,  0., 27., 48., 33.,
       83., 30., 69., 90., 81., 84., 88., 18., 11., 85., 93., 34., 89.,
        2., 70., 37., 43.,  8., 35., 99., 55., 75., 51., 92., 29., 63.,
       61., 46., 41., 31., 45., 67., 22., 23.,  9., 72., 96., 58., 66.,
       73., 39., 19., 97., 52., 76., 78., 80., 74., 24.])

In [62]:
data["months_since_policy_inception"].mode()

0    59.0
Name: months_since_policy_inception, dtype: float64

In [63]:
data["months_since_policy_inception"]=data["months_since_policy_inception"].fillna(value="59.0")

We check the column "number_of_open_complaints", to see how to treat the missing values.

In [64]:
data["number_of_open_complaints"].unique()

array([nan,  0.,  2.,  1.,  3.,  5.,  4.])

In [65]:
data["number_of_open_complaints"].mode()

0    0.0
Name: number_of_open_complaints, dtype: float64

In [66]:
data["number_of_open_complaints"]=data["number_of_open_complaints"].fillna(value="0.0")

We check the column "number_of_policies", to see how to treat the missing values.

In [67]:
data["number_of_policies"].unique()

array([nan,  1.,  8.,  2.,  7.,  9.,  4.,  3.,  6.,  5.])

In [68]:
data["number_of_policies"].mode()

0    1.0
Name: number_of_policies, dtype: float64

In [69]:
data["number_of_policies"]=data["number_of_policies"].fillna(value="1.0")

We check the column "total_claim_amount", to see how to treat the missing values.

In [70]:
data["total_claim_amount"].unique()

array([        nan,  384.811147, 1131.464935, ...,  541.282007,
        198.234764,  790.784983])

In [71]:
data["total_claim_amount"].mean()

434.0887943128969

In [72]:
data["total_claim_amount"]=data["total_claim_amount"].fillna(value="434.0887943128969")

We define a new way of expressing the values inside the column using numbers from zero to four.
We order the from smallest number of ocurrences to largest

In [73]:
#data["state"].unique()

In [74]:
#data["state"].value_counts().nlargest(5)

In [75]:
#data["state"] = data["state"].map({"Washington" : 0, "Nevada" : 1, "Arizona" : 2,"Oregon":3, "California":4 })

In [76]:
data

,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,renew_offer_type,...,vehicle_class,vehicle_size,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,Washington,No,0,Bachelor,Employed,F,Suburban,Married,C,Offer1,...,TDC,Medsize,2004.350666,37657.38000875848,93.21929056273265,93.21929056273265,59.0,0.0,1.0,434.0887943128969
1,Arizona,No,1,Bachelor,Unemployed,F,Suburban,Single,P,Offer3,...,NaN,Medsize,2004.350666,37657.38000875848,93.21929056273265,93.21929056273265,59.0,0.0,1.0,434.0887943128969
2,Nevada,No,2,Bachelor,Employed,F,Suburban,Married,P,Offer1,...,TDC,Medsize,2004.350666,37657.38000875848,93.21929056273265,93.21929056273265,59.0,0.0,1.0,434.0887943128969
3,California,No,0,Bachelor,Unemployed,M,Suburban,Married,C,Offer1,...,SUV,Medsize,2004.350666,37657.38000875848,93.21929056273265,93.21929056273265,59.0,0.0,1.0,434.0887943128969
4,Washington,No,0,Bachelor,Employed,M,Rural,Single,P,Offer1,...,NaN,Medsize,2004.350666,37657.38000875848,93.21929056273265,93.21929056273265,59.0,0.0,1.0,434.0887943128969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18263,California,Unknown,0,Unknown,Unknown,F,Suburban,Unknown,O,other,...,O,Medsize,23405.98798,71941.0,73.0,18.0,89.0,0.0,2.0,198.234764
18264,California,Unknown,0,Unknown,Unknown,F,Suburban,Unknown,O,other,...,O,Medsize,3096.511217,21604.0,79.0,14.0,28.0,0.0,1.0,379.2
18265,California,Unknown,0,Unknown,Unknown,F,Suburban,Unknown,O,other,...,O,Medsize,8163.890428,0.0,85.0,9.0,37.0,3.0,2.0,790.784983
18266,California,Unknown,0,Unknown,Unknown,F,Suburban,Unknown,O,other,...,O,Medsize,7524.442436,21941.0,96.0,34.0,3.0,0.0,3.0,691.2


In [77]:
from sklearn.model_selection import train_test_split

In [78]:
#First we drop the target column (total claim amount)
X=data.drop('total_claim_amount', axis=1)
y=data['total_claim_amount']

In [79]:
#Let's check their shape
X.shape

(18268, 20)

In [80]:
X["customer_lifetime_value"]=X["customer_lifetime_value"].astype(float)

In [81]:
X["income"]=X["income"].astype(float)

In [82]:
X["monthly_premium_auto"]=X["monthly_premium_auto"].astype(float)

In [83]:
X["months_since_last_claim"]=X["months_since_last_claim"].astype(float)

In [84]:
X["months_since_policy_inception"]=X["months_since_policy_inception"].astype(float)

In [85]:
X["number_of_open_complaints"]=X["number_of_open_complaints"].astype(float)

In [86]:
X["number_of_policies"]=X["number_of_policies"].astype(float)

In [87]:
y.shape

(18268,)

In [88]:
#X_train and X_test split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [89]:
#Define categorical and numerical dataframes
#numerical_col=X.select_dtypes(include=[np.number])
#categorical_col=X.select_dtypes(exclude=[np.number])

In [90]:
#Split X_train and X_test
X_train_cat=X_train.select_dtypes(exclude=[np.number])
X_train_num=X_train.select_dtypes(include=[np.number])
X_test_cat=X_test.select_dtypes(exclude=[np.number])
X_test_num=X_test.select_dtypes(include=[np.number])


In [91]:
X.dtypes


state                             object
response                          object
coverage                           int64
education                         object
employmentstatus                  object
gender                            object
location_code                     object
marital_status                    object
policy_type                       object
renew_offer_type                  object
sales_channel                     object
vehicle_class                     object
vehicle_size                      object
customer_lifetime_value          float64
income                           float64
monthly_premium_auto             float64
months_since_last_claim          float64
months_since_policy_inception    float64
number_of_open_complaints        float64
number_of_policies               float64
dtype: object

In [92]:
X_train_num

,coverage,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
14727,0,29049.713150,21070.000000,195.000000,6.000000,47.0,0.0,2.0
2088,0,2004.350666,37657.380009,93.219291,93.219291,59.0,0.0,1.0
14334,0,8923.113112,0.000000,125.000000,22.000000,80.0,0.0,9.0
6024,1,2004.350666,37657.380009,93.219291,93.219291,59.0,0.0,1.0
14086,0,4904.894731,12902.000000,139.000000,3.000000,51.0,0.0,1.0
...,...,...,...,...,...,...,...,...
11284,0,15886.031360,56362.000000,197.000000,30.000000,47.0,1.0,6.0
11964,0,9863.502214,99654.000000,121.000000,13.000000,95.0,0.0,3.0
5390,1,2004.350666,37657.380009,93.219291,93.219291,59.0,0.0,1.0
860,1,2004.350666,37657.380009,93.219291,93.219291,59.0,0.0,1.0


In [93]:
X_train_num["coverage"].unique()

array([0, 1, 2], dtype=int64)

In [94]:
X_train_cat

,state,response,education,employmentstatus,gender,location_code,marital_status,policy_type,renew_offer_type,sales_channel,vehicle_class,vehicle_size
14727,California,Unknown,Unknown,Unknown,F,Suburban,Unknown,O,other,other,O,Medsize
2088,California,No,Bachelor,Medical Leave,F,Suburban,Married,P,Offer1,Branch,TDC,Small
14334,California,Unknown,Unknown,Unknown,F,Suburban,Unknown,O,other,other,O,Medsize
6024,Nevada,No,High School or Below,Employed,F,Urban,Single,P,Offer3,Call Center,NaN,Medsize
14086,California,Unknown,Unknown,Unknown,F,Suburban,Unknown,O,other,other,O,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...
11284,California,Unknown,Unknown,Unknown,F,Suburban,Unknown,O,other,other,O,Medsize
11964,California,Unknown,Unknown,Unknown,F,Suburban,Unknown,O,other,other,O,Medsize
5390,California,No,High School or Below,Unemployed,F,Suburban,Single,P,Offer1,Call Center,SUV,Medsize
860,Arizona,Yes,Bachelor,Unemployed,F,Suburban,Divorced,P,Offer1,Call Center,SUV,Medsize


Encoding of categorical variables (OneHotEncoder)

We perform fit-transform only on the train set to avoid train-test contamination

In [95]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown="ignore",sparse_output=False).set_output(transform='pandas')
ohetransform=ohe.fit_transform(X_train_cat)
cols = ohe.get_feature_names_out()

In [96]:
ohetransform=ohetransform.reset_index(drop=True)

In [97]:
ohetransform

,state_Arizona,state_California,state_Nevada,state_Oregon,state_Washington,response_No,response_Unknown,response_Yes,education_Bachelor,education_College,...,sales_channel_Web,sales_channel_other,vehicle_class_Luxury,vehicle_class_O,vehicle_class_SUV,vehicle_class_TDC,vehicle_class_nan,vehicle_size_Large,vehicle_size_Medsize,vehicle_size_Small
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14609,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
14610,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
14611,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
14612,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [98]:
ohetransform.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14614 entries, 0 to 14613
Data columns (total 51 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   state_Arizona                   14614 non-null  float64
 1   state_California                14614 non-null  float64
 2   state_Nevada                    14614 non-null  float64
 3   state_Oregon                    14614 non-null  float64
 4   state_Washington                14614 non-null  float64
 5   response_No                     14614 non-null  float64
 6   response_Unknown                14614 non-null  float64
 7   response_Yes                    14614 non-null  float64
 8   education_Bachelor              14614 non-null  float64
 9   education_College               14614 non-null  float64
 10  education_Doctor                14614 non-null  float64
 11  education_High School or Below  14614 non-null  float64
 12  education_Master                

OHE  (only transform)  on test data

In [99]:
X_test_cat = ohe.transform(X_test_cat)

Normalization of numerical values (MinMax scaler)

Transformation on train and test data

In [100]:
column_names = X_train_num.columns.values.tolist()

In [101]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
X_train_num_scaled = scaler.fit_transform(X_train_num)


In [102]:
X_train_num_scaled

array([[0.        , 0.3334469 , 0.21078221, ..., 0.47474747, 0.        ,
        0.125     ],
       [0.        , 0.00130599, 0.37672072, ..., 0.5959596 , 0.        ,
        0.        ],
       [0.        , 0.08627449, 0.        , ..., 0.80808081, 0.        ,
        1.        ],
       ...,
       [0.5       , 0.00130599, 0.37672072, ..., 0.5959596 , 0.        ,
        0.        ],
       [0.5       , 0.00130599, 0.37672072, ..., 0.5959596 , 0.        ,
        0.        ],
       [0.        , 0.10312819, 0.24732646, ..., 0.        , 0.        ,
        0.5       ]])

In [103]:
X_train_numsc_df= pd.DataFrame(data=X_train_num_scaled, columns=column_names)

In [104]:

X_train_concat = pd.concat((X_train_numsc_df,ohetransform),axis=1)

In [105]:
X_train_numsc_df

,coverage,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,0.0,0.333447,0.210782,0.565401,0.064364,0.474747,0.0,0.125
1,0.0,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000
2,0.0,0.086274,0.000000,0.270042,0.236003,0.808081,0.0,1.000
3,0.5,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000
4,0.0,0.036927,0.129070,0.329114,0.032182,0.515152,0.0,0.000
...,...,...,...,...,...,...,...,...
14609,0.0,0.171785,0.563840,0.573840,0.321822,0.474747,0.2,0.625
14610,0.0,0.097823,0.996929,0.253165,0.139456,0.959596,0.0,0.250
14611,0.5,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000
14612,0.5,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000


In [106]:
X_train_concat

,coverage,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state_Arizona,state_California,...,sales_channel_Web,sales_channel_other,vehicle_class_Luxury,vehicle_class_O,vehicle_class_SUV,vehicle_class_TDC,vehicle_class_nan,vehicle_size_Large,vehicle_size_Medsize,vehicle_size_Small
0,0.0,0.333447,0.210782,0.565401,0.064364,0.474747,0.0,0.125,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.086274,0.000000,0.270042,0.236003,0.808081,0.0,1.000,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.5,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.036927,0.129070,0.329114,0.032182,0.515152,0.0,0.000,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14609,0.0,0.171785,0.563840,0.573840,0.321822,0.474747,0.2,0.625,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
14610,0.0,0.097823,0.996929,0.253165,0.139456,0.959596,0.0,0.250,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
14611,0.5,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
14612,0.5,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [107]:
X_train_concat.columns = X_train_concat.columns.astype(str)

In [108]:
X_test_concat = pd.concat((X_test_num,X_test_cat),axis=1)

In [109]:
X_test_concat.columns = X_test_concat.columns.astype(str)

In [110]:
X_test_concat

,coverage,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state_Arizona,state_California,...,sales_channel_Web,sales_channel_other,vehicle_class_Luxury,vehicle_class_O,vehicle_class_SUV,vehicle_class_TDC,vehicle_class_nan,vehicle_size_Large,vehicle_size_Medsize,vehicle_size_Small
4084,0,2004.350666,37657.380009,93.219291,93.219291,59.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1570,0,2004.350666,37657.380009,93.219291,93.219291,59.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4525,1,2004.350666,37657.380009,93.219291,93.219291,59.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3310,2,2004.350666,37657.380009,93.219291,93.219291,59.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
15737,0,2489.662064,0.000000,74.000000,3.000000,64.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7539,0,2004.350666,37657.380009,93.219291,93.219291,59.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2126,0,2004.350666,37657.380009,93.219291,93.219291,59.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
12493,0,4944.088762,53607.000000,61.000000,4.000000,18.0,0.0,9.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6306,0,2004.350666,37657.380009,93.219291,93.219291,59.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [111]:
X_train_concat

,coverage,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state_Arizona,state_California,...,sales_channel_Web,sales_channel_other,vehicle_class_Luxury,vehicle_class_O,vehicle_class_SUV,vehicle_class_TDC,vehicle_class_nan,vehicle_size_Large,vehicle_size_Medsize,vehicle_size_Small
0,0.0,0.333447,0.210782,0.565401,0.064364,0.474747,0.0,0.125,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.086274,0.000000,0.270042,0.236003,0.808081,0.0,1.000,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.5,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.036927,0.129070,0.329114,0.032182,0.515152,0.0,0.000,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14609,0.0,0.171785,0.563840,0.573840,0.321822,0.474747,0.2,0.625,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
14610,0.0,0.097823,0.996929,0.253165,0.139456,0.959596,0.0,0.250,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
14611,0.5,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
14612,0.5,0.001306,0.376721,0.135946,1.000000,0.595960,0.0,0.000,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [112]:
X_train_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14614 entries, 0 to 14613
Data columns (total 59 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   coverage                        14614 non-null  float64
 1   customer_lifetime_value         14614 non-null  float64
 2   income                          14614 non-null  float64
 3   monthly_premium_auto            14614 non-null  float64
 4   months_since_last_claim         14614 non-null  float64
 5   months_since_policy_inception   14614 non-null  float64
 6   number_of_open_complaints       14614 non-null  float64
 7   number_of_policies              14614 non-null  float64
 8   state_Arizona                   14614 non-null  float64
 9   state_California                14614 non-null  float64
 10  state_Nevada                    14614 non-null  float64
 11  state_Oregon                    14614 non-null  float64
 12  state_Washington                

In [113]:
from sklearn.linear_model import LinearRegression

In [114]:
lr=LinearRegression()

In [115]:
#Train the model on the training set
lr.fit(X_train_concat,y_train)

LinearRegression()

In [116]:
y_pred=lr.predict(X_test_concat)

In [117]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [118]:
r2=r2_score(y_test,y_pred)

In [119]:
mse=mean_squared_error(y_test,y_pred)

In [120]:
mae=mean_absolute_error(y_test,y_pred)

In [121]:
intercept=lr.intercept_

In [122]:
print("The r2 score is" , r2)
print("The mse score is", mse)
print("The mae score is", mae)
print("The intercept of the regression is", intercept)

The r2 score is -4909232465.412144
The mse score is 206088424927910.4
The mae score is 12442612.458916083
The intercept of the regression is 1241273278281916.8


In [123]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [125]:
nn = MLPRegressor(
    activation='relu',
    hidden_layer_sizes=(10, 100),
    alpha=0.001,
    random_state=20,
    early_stopping=False
)

In [133]:
ohe2 = OneHotEncoder(handle_unknown="ignore",sparse_output=False).set_output(transform='pandas')
ohetransform2=ohe2.fit_transform(X_train)
cols = ohe2.get_feature_names_out()
ohetransform2=ohetransform2.reset_index(drop=True)
X_train_2=ohetransform2



In [134]:
nn.fit(X_train_2, y_train)

MLPRegressor(alpha=0.001, hidden_layer_sizes=(10, 100), random_state=20)

In [137]:
X_train_2.shape

(12787, 10192)

In [129]:
y_train.shape

(12787,)

In [135]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [ ]:
pred = nn.predict(X_test)

In [ ]:
test_set_rsquared = nn.score(X_test, y_test)
test_set_rmse = np.sqrt(mean_squared_error(y_test, pred))

In [ ]:
print('R_squared value: ', test_set_rsquared)
print('RMSE: ', test_set_rmse)

As we are having problems with the X_train_concat matrix, as KNN is not acepting some values, we will encode it again.

In [141]:
%%time
from sklearn.neighbors import KNeighborsRegressor
KNN=KNeighborsRegressor(n_neighbors=3)
KNN.fit(X_train_2,y_train)




CPU times: total: 2.53 s
Wall time: 3.75 s


KNeighborsRegressor(n_neighbors=3)

In [ ]:
%%time
score=KNN.score(X_test,y_test)
score

In [ ]:
%%time
score=KNN.score(X_train_2,y_train)
score

As we are having problems with the X_train_concat matrix, as KNN is not acepting some values, we will encode it again.

In [ ]:

ohe2 = OneHotEncoder(handle_unknown="ignore",sparse_output=False).set_output(transform='pandas')
ohetransform2=ohe2.fit_transform(X_train_concat)
cols = ohe2.get_feature_names_out()
ohetransform2=ohetransform2.reset_index(drop=True)
X_train_concat2=ohetransform2
X_train_concat2.columns = X_train_concat.columns
